# 【开源实习】针对OpenDataLab社区“万卷·丝路” 开源库中的韩语语种的多模态数据资源，开发可在昇腾AI开发板上运行的应用，开发可在香橙派AIpro开发板运行的应用


## 环境准备
开发者拿到香橙派开发板后，首先需要进行硬件资源确认，镜像烧录及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

开发板：香橙派Aipro或其他同硬件开发板  
开发板镜像: Ubuntu镜像  
CANN Toolkit/Kernels：8.1.RC1  
MindSpore: 2.6.0  
MindSpore NLP: 0.4.1  
Python: 3.9

### 镜像烧录
运行该案例需要烧录香橙派官网ubuntu镜像，烧录流程参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--镜像烧录](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html) 章节。

### CANN升级
CANN升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--CANN升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。

### MindSpore升级
MindSpore升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--MindSpore升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。

In [1]:
from mindnlp.transformers import Qwen2VLForConditionalGeneration, Qwen2VLConfig, AutoProcessor
from qwen_vl_utils import process_vision_info
import mindspore 

path = "/home/HwHiAiUser/Qwen2-VL-2B-Instruct"
# path = "/home/HwHiAiUser/qwen2ko_final"
# path_weight = "/home/HwHiAiUser/.cache/modelscope/hub/models/Qwen/Qwen2-VL-2B-Instruct/test.ckpt"
# default: Load the model on the available device(s)
# config = Qwen2VLConfig.from_pretrained(path, ms_dtype=mindspore.float16)
# model = Qwen2VLForConditionalGeneration(config)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    path, ms_dtype=mindspore.float16,
    trust_remote_code = True
)
# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

[WARNING] ME(5415:255086263164960,MainProcess):2025-10-27-10:21:32.313.68 [mindspore/run_check/_check_version.py:324] MindSpore version 2.5.0 and Ascend AI software package (Ascend Data Center Solution)version 7.7 does not match, the version of software package expect one of ['7.5', '7.6']. Please refer to the match info on: https://www.mindspore.cn/install
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, get

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# default processer
# processor = AutoProcessor.from_pretrained(path, ms_dtype=mindspore.float16)

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(path, min_pixels=min_pixels, max_pixels=max_pixels, ms_dtype=mindspore.float16)


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(type(text))
text

# inputs = inputs.to("cuda")

<class 'str'>


'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

In [4]:
image_inputs, video_inputs = process_vision_info(messages)
image_inputs

[<PIL.Image.Image image mode=RGB size=2044x1372>]

In [5]:
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="ms",
)
inputs

{'input_ids': Tensor(shape=[1, 1272], dtype=Int64, value=
[[151644,   8948,    198 ... 151644,  77091,    198]]), 'attention_mask': Tensor(shape=[1, 1272], dtype=Int64, value=
[[1, 1, 1 ... 1, 1, 1]]), 'pixel_values': Tensor(shape=[4988, 1176], dtype=Float32, value=
[[ 8.50052655e-01,  8.64651084e-01,  8.64651084e-01 ...  1.42067373e+00,  1.43489385e+00,  1.43489385e+00],
 [ 1.01063538e+00,  1.01063538e+00,  1.02523375e+00 ...  1.50599408e+00,  1.50599408e+00,  1.52021420e+00],
 [ 9.96036947e-01,  1.01063538e+00,  1.02523375e+00 ...  1.50599408e+00,  1.52021420e+00,  1.52021420e+00],
 ...
 [-2.44829327e-01, -2.59427756e-01, -2.88624585e-01 ... -1.86193749e-01, -1.71973675e-01, -1.86193749e-01],
 [-1.42640352e-01, -1.13443494e-01, -5.50497957e-02 ... -1.29313484e-01, -1.43533543e-01, -1.71973675e-01],
 [-2.44829327e-01, -2.44829327e-01, -2.59427756e-01 ... -3.85274649e-01, -4.27934855e-01, -4.56374973e-01]]), 'image_grid_thw': Tensor(shape=[1, 3], dtype=Int64, value=
[[ 1, 58, 86]])}

In [7]:
# 不修改成int32类型会报错
for k in ['input_ids', 'attention_mask', 'position_ids']:
    if k in inputs and inputs[k].dtype == mindspore.int64:
        print(k)
        inputs[k] = inputs[k].astype(mindspore.int32)
# 只改“索引类”输入为 int32；mask 用浮点
# if 'input_ids' in inputs:
#     inputs['input_ids'] = inputs['input_ids'].astype(mindspore.int32)

# if 'position_ids' in inputs:
#     inputs['position_ids'] = inputs['position_ids'].astype(mindspore.int64)

# if 'attention_mask' in inputs:
#     # 用 float16（或 float32 也可），很多注意力实现期望 mask 是浮点参与加/乘
#     inputs['attention_mask'] = inputs['attention_mask'].astype(mindspore.float16)
# inputs

{'input_ids': Tensor(shape=[1, 1272], dtype=Int32, value=
[[151644,   8948,    198 ... 151644,  77091,    198]]), 'attention_mask': Tensor(shape=[1, 1272], dtype=Float16, value=
[[ 1.0000e+00,  1.0000e+00,  1.0000e+00 ...  1.0000e+00,  1.0000e+00,  1.0000e+00]]), 'pixel_values': Tensor(shape=[4988, 1176], dtype=Float32, value=
[[ 8.50052655e-01,  8.64651084e-01,  8.64651084e-01 ...  1.42067373e+00,  1.43489385e+00,  1.43489385e+00],
 [ 1.01063538e+00,  1.01063538e+00,  1.02523375e+00 ...  1.50599408e+00,  1.50599408e+00,  1.52021420e+00],
 [ 9.96036947e-01,  1.01063538e+00,  1.02523375e+00 ...  1.50599408e+00,  1.52021420e+00,  1.52021420e+00],
 ...
 [-2.44829327e-01, -2.59427756e-01, -2.88624585e-01 ... -1.86193749e-01, -1.71973675e-01, -1.86193749e-01],
 [-1.42640352e-01, -1.13443494e-01, -5.50497957e-02 ... -1.29313484e-01, -1.43533543e-01, -1.71973675e-01],
 [-2.44829327e-01, -2.44829327e-01, -2.59427756e-01 ... -3.85274649e-01, -4.27934855e-01, -4.56374973e-01]]), 'image_grid_thw'

In [8]:
mindspore.set_context(
    mode=mindspore.GRAPH_MODE,
    device_target='Ascend',
    enable_graph_kernel=False      # ← 关键：关闭 GraphKernel
    # ascend_config=dict(enable_op_layout=False)  # ← 关键：禁止自动改 Layout
)
print(mindspore.get_context('device_target'))

[WARNING] ME(63017:255086478434336,MainProcess):2025-10-26-20:18:49.258.425 [mindspore/context.py:1335] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.
[WARNING] ME(63017:255086478434336,MainProcess):2025-10-26-20:18:49.267.082 [mindspore/context.py:1992] For 'context.set_context', 'enable_graph_kernel' parameter is deprecated, and will be removed in the next version. Please use jit_config={'jit_level': 'O1'} instead.


Ascend


In [9]:
print(type(inputs))

<class 'mindnlp.transformers.feature_extraction_utils.BatchFeature'>


In [11]:
generated_ids = model.generate(**inputs, max_new_tokens=16)

RuntimeError: aclnnCatGetWorkspaceSize call failed, please check!

----------------------------------------------------
- Ascend Error Message:
----------------------------------------------------
EZ1001: [PID: 63017] 2025-10-26-20:22:02.088.280 tensor 0 not implemented for DT_INT32, should be in dtype support list [].[THREAD:63395]

(Please search "CANN Common Error Analysis" at https://www.mindspore.cn for error code description)

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ops/kernel/ascend/pyboost/auto_generate/pyboost_ascend_ops_2.cc:2201 operator()


# Inference: Generation of the output

generated_ids = model.generate(**inputs, max_new_tokens=64)


In [10]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

NameError: name 'generated_ids' is not defined